In [1]:
%pwd

'd:\\Projects\\ML Projects\\End-to-End Wine Quality\\End-to-End-ML-Project\\research'

In [2]:
import os

In [3]:
os.chdir('../')
%pwd

'd:\\Projects\\ML Projects\\End-to-End Wine Quality\\End-to-End-ML-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    target_column: str

In [5]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            target_column=schema.name
        )
        return model_trainer_config

In [6]:
import pandas as pd
import os
from MLProject import logger
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.metrics import classification_report

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        rfc = RandomForestClassifier(random_state=101)
        rfc.fit(train_x, train_y.values.ravel(), )
        pred_y = rfc.predict(test_x)
        report = classification_report(test_y, pred_y)
        print(report)
        logger.info(f"Model training completed.")

        joblib.dump(rfc, os.path.join(self.config.root_dir, self.config.model_name))
        logger.info(f"Model saved.")
        

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_training = ModelTrainer(config=model_trainer_config)
    model_training.train()
except Exception as e:
    raise e

[2025-04-27 14:04:25,730: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2025-04-27 14:04:25,736: INFO: common: yaml file: params.yaml loaded successfully.]
[2025-04-27 14:04:25,738: INFO: common: yaml file: schema.yaml loaded successfully.]
[2025-04-27 14:04:25,741: INFO: common: Created directory at: artifacts]
[2025-04-27 14:04:25,741: INFO: common: Created directory at: artifacts/model_trainer]
              precision    recall  f1-score   support

           0       0.79      0.78      0.79       109
           1       0.80      0.82      0.81       120

    accuracy                           0.80       229
   macro avg       0.80      0.80      0.80       229
weighted avg       0.80      0.80      0.80       229

[2025-04-27 14:04:26,413: INFO: 4495762: Model training completed.]
[2025-04-27 14:04:26,454: INFO: 4495762: Model saved.]
